In [1]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix
#git init
#git config user.name "someone"
#git config user.email "someone@someplace.com"
#git add *
#git commit -m "some init msg"

In [3]:
C_X0 = 0.1  # gX/L or 1e-4 mass fraction
C_S0 = 100  # gS/L or 0.1 mass fraction
Y = 0.4     # kgX/kgS
mu_max_hour = 0.1  # 1/h
k_d_hour = 0.01 # 1/h
K_S = 3.76    # kgS/m3
C_S_in = 1560 # kg/m3 (pure glucose in)
V_0 = 202   # ml
F_0 = 0     # ml/s
activation = 0
set_point = 0.25     # kg/m3
Yp = 0.15 #kgP/kgS
Ypace = 0.15 #kgP/kgS
C_P0 = 0 #gP/L
C_Pace0 = 0 #gP/L

In [4]:
def BioReactor(tf,C_X0,C_S0,Y,mu_max_hour,k_d_hour,K_S,C_S_in,C_NAOH_in,V_0,F_0, Fin_S, Fin_NAOH, t_act_S, t_act_NAOH, Yp,C_NAOH0, C_P0, C_Pace0,Ypace,pH0):
    t0 = 0
    dt = 1
    tf=tf*3600
    molarmass_acid=60.05
    molarmass_NAOH=39.997
    Ka_acid=1.8*10**-5
    Kb_ace=5.6*10**-10
    C_X = C_X0
    C_S = C_S0
    C_P =C_P0
    C_Pace =C_Pace0
    C_NAOH=C_NAOH0
    pH =pH0
    t = t0
    V = V_0/10**6
    F = F_0/10**6
    F_S = F_0/10**6
    F_NAOH = F_0/10**6
    mu_max=mu_max_hour/3600
    k_d=k_d_hour/3600
    activation=0
    activation_NAOH=0
    check=-1
    warning=0

    C_X_v, C_S_v, t_v, V_v, F_v, F_S_v,F_NAOH_v, C_P_v, C_Pace_v, C_NAOH_v, pH_v = [C_X], [C_S], [t], [V], [F],[F_S],[F_NAOH],[C_P],[C_Pace],[C_NAOH],[pH]
    mu_v, rX_v, rS_v, rXg_v, rXd_v = [], [], [], [], []
    while t < tf:
        check=check+1
        # Check if there is activation of the pump1
        if t % (t_act_S*3600) == 0 :
            activation = 1
            F_S  = Fin_S/10**6  # kg/m3/s
        else:
            activation = 0
            F_S = 0
            
        if t % (t_act_NAOH*3600) == 0 and check>0:
            activation_NAOH = 1
            F_NAOH  = Fin_NAOH/10**6  # kg/m3/s
        else:
            activation = 0
            F_NAOH = 0
        F=F_S+F_NAOH
        

        # Compute rates
        mu = mu_max*C_S/(C_S+K_S)
        rXg = mu * C_X
        rXd = k_d * C_X
        rX = rXg - rXd
        rS = -1/Y * rXg
        rP = -Yp * rS
        rPace = -Ypace * rS

        # Compute derivatives
        dXdt = rX-F/V*C_X
        dSdt = (F_S/V*(C_S_in-C_S))+rS-(F_NAOH/V*C_S)
        dVdt = F
        dPdt = rP-F/V*C_P
        dPacedt = rPace-F/V*C_Pace
        dNAOHdt =(F_NAOH/V*(C_NAOH_in-C_NAOH))-F_S/V*C_NAOH
        
        
        # Update results
        t += dt
        C_X += dXdt*dt
        C_S += dSdt*dt
        V += dVdt*dt
        C_P += dPdt*dt
        C_Pace += dPacedt*dt
        C_NAOH+=dNAOHdt*dt
        C_ph=C_Pace/molarmass_acid
        #OH-
        C_pH_NAOH=C_NAOH/molarmass_NAOH
        #H3O+
        sqrt1=math.sqrt(Ka_acid*C_ph)
        sqrt2=math.sqrt(Kb_ace*C_ph)
        
        if C_ph==0 and C_NAOH==0:
            pH=7
        elif C_ph==0 and C_NAOH!=0:
            pH=14-math.log10(C_pH_NAOH)
        elif C_ph!=0 and C_NAOH==0:
            pH=-1*math.log10(sqrt1)
        elif C_pH_NAOH>C_ph:
            pH=14-1*math.log10(C_pH_NAOH-C_ph)
        elif C_pH_NAOH==C_ph:
            pH=14-math.log10(sqrt)
        else:
            pH=-1*math.log10(Ka_acid)+math.log10(C_pH_NAOH/sqrt1)
        
        if C_pH_NAOH>C_ph:
            warning=warning+1
        else:
            pass
                
        # Store results
        C_X_v.append(C_X)
        C_S_v.append(C_S)
        C_P_v.append(C_P)
        C_Pace_v.append(C_Pace)
        C_NAOH_v.append(C_NAOH)
        pH_v.append(pH)
        t_v.append(t)
        V_v.append(V*10**6)
        F_v.append(F*10**6)
        F_S_v.append(F_S*10**6)   
        F_NAOH_v.append(F_NAOH*10**6)   
        mu_v.append(mu)
        rS_v.append(rS)
        rX_v.append(rX)
        rXg_v.append(rXg)
        rXd_v.append(rXd)
        
    #mu_v=mu_v[::3600]
    if warning>0:
        title_ph="\n WARNING: NaOH was added in excessive quantaties pH values might be incorrect"
    else:
        title_ph=" "
    
    t_plot = [t/24/3600 for t in t_v]
    mu_plot = [mu*3600 for mu in mu_v]
    fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2,figsize=(15,15))
    ax1.plot(t_plot,C_S_v,'r-',t_plot,C_X_v, t_plot,C_P_v,t_plot,C_Pace_v,'g-',t_plot,C_NAOH_v,'m-',linewidth=2)
    ax1.legend(['Concentration glucose','Concentration biomass','Concentration ethanol','Concentration acetate','Concentration NaOH'])
    ax1.set(xlabel='Time (days)', ylabel='Concentration (kg/$m^3$)',title='Concentration of different compounds in N.I.N.O.')

    ax2.plot(t_plot, pH_v,linewidth=2)
    ax2.plot([0,tf/(3600*24)],[6,6], color='r')
    ax2.plot([0,tf/(3600*24)],[8,8], color='r')
    ax2.set(xlabel='Time (days)', ylabel='pH', title='pH in the reactor over time'+title_ph)
    ax2.legend(['pH of reactor','pH threshold'])

    ax3.plot(t_plot[:-1],mu_plot,'y-',linewidth=2)
    ax3.set(xlabel='Time (days)', ylabel='$\mu$ (1/h)', title='Growth rate over time')
    
    ax4.set_title('Volume and flowrate of N.I.N.O')
    ax4.scatter(t_plot,F_v,linewidth=2,label='Flowrate') 
    ax4.set(xlabel='Time (days)', ylabel='Flowrate (ml/s)')
    ax4.legend(loc=6)
    ax5=ax4.twinx()
    ax5.plot(t_plot[1:],V_v[1:],'g-',linewidth=2,label='Volume')
    ax5.set_ylabel('Volume (mL)')
    ax5.ticklabel_format(axis="y")
    ax5.legend(loc=5)
    #lns = lns1+lns2
    #labs = [l.get_label() for l in lns]
    #ax4.legend(lns,labs,loc=0)


In [5]:
file = open("Project_nino.jpg", "rb")
image = file.read()

image_headline = widgets.Image(
                    value=image,
                    format='jpg',
                    width='250'
                )
display(image_headline)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00\xc0\x00\xc0\x00\x00\xff\xe1\x00"Exif\x00\x00MM\…

In [6]:
file = open("introduction.txt", "r")
content = file.read()
print(content)

Welcome to the Growth calculator of Project N.I.N.O.! 
This growth calculator is used to research the playability of a certain design. 
The calculator will project the most important parameters of growth of the micro organism.
Fill in the parameters below to find the best biological design for the N.I.N.O.


In [7]:
style = {'description_width': 'initial'}

In [12]:
Bioreactor_final = widgets.interactive(BioReactor, tf=widgets.BoundedFloatText(value=24, min=0, max=3600*24*10,step=1, description='Total time (h):', style=style, disabled=False),
                                       C_X0=widgets.BoundedFloatText(value=1, min=0, max=100.0,step=0.1, description='Biomass concentration (g/L):',style=style, disabled=False),
                                       C_S0=widgets.BoundedFloatText(value=0, min=0, max=100.0,step=0.1, description='Concentration glucose (g/L):',style=style, disabled=False),
                                       Y=widgets.BoundedFloatText(value=0.4, min=0, max=1.0,step=0.1, description='Yield:',style=style, disabled=False), 
                                       mu_max_hour=widgets.BoundedFloatText(value=0.3, min=0, max=10.0,step=0.01, description='mu_max (1/h):',style=style, disabled=False),
                                       K_S=widgets.BoundedFloatText(value=3.6, min=0, max=10.0,step=0.01, description='K_S (kg s/m3):',style=style, disabled=False),
                                       C_S_in=widgets.BoundedFloatText(value=500, min=0, max=910,step=1, description='glucose concentration in feed (kg/m3):',style=style, disabled=False),
                                       C_NAOH_in=widgets.BoundedFloatText(value=30, min=0, max=910,step=1, description='NaOH concentration in feed (kg/m3):',style=style, disabled=False),
                                       V_0=widgets.BoundedFloatText(value=50, min=0, max=100,step=1, description='Volume (ml):',style=style, disabled=False),
                                       F_0=widgets.BoundedFloatText(value=0, min=0, max=5,step=0.1, description='Feed volume (ml):',style=style, disabled=False),
                                       k_d_hour=widgets.BoundedFloatText(value=0.01, min=0, max=0.1,step=0.001, description='K_d (1/h):',style=style, disabled=False),
                                       Fin_S=widgets.BoundedFloatText(value=0.5, min=0, max=5,step=0.01, description='Feed glucose solution in (ml):',style=style, disabled=False),
                                       Fin_NAOH=widgets.BoundedFloatText(value=0.5, min=0, max=5,step=0.01, description='Feed NaOH solution in (ml):',style=style, disabled=False),
                                       t_act_S=widgets.BoundedFloatText(value=1, min=0, max=20,step=0.1, description='Activation interval substrate(h):',style=style, disabled=False),
                                       t_act_NAOH=widgets.BoundedFloatText(value=1, min=0, max=20,step=0.1, description='Activation interval base (h):',style=style, disabled=False),
                                       Yp=widgets.BoundedFloatText(value=0.4, min=0, max=1.0,step=0.1, description='Yield (P/S):',style=style, disabled=False),
                                       C_NAOH0=widgets.BoundedFloatText(value=0.1, min=0, max=100.0,step=0.1, description='Concentration NaOH (g/L):',style=style, disabled=False),
                                       C_P0=widgets.BoundedFloatText(value=0, min=0, max=100.0,step=0.1, description='Concentration ethanol (g/L):',style=style, disabled=False),
                                       Ypace=widgets.BoundedFloatText(value=0.12, min=0, max=1.0,step=0.01, description='Yield (actetate/S):',style=style, disabled=False),
                                       C_Pace0=widgets.BoundedFloatText(value=1, min=0, max=100.0,step=0.1, description='Concentration acetate (g/L):',style=style, disabled=False),
                                       pH0=widgets.BoundedFloatText(value=7, min=0, max=14.0,step=0.1, description='pH:',style=style, disabled=False) );

In [13]:
controls = widgets.HBox(Bioreactor_final.children[:-1], layout = widgets.Layout(flex_flow='row wrap'))
output = Bioreactor_final.children[-1]
display(widgets.VBox([controls, output]))